In [1]:
from mica_text_coref.coref.seq_coref import data_util

import collections
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

Check if label ids have correct labels

In [2]:
train_dataset = data_util.load_tensors(
    ("/home/sbaruah_usc_edu/mica_text_coref/data/tensors/"
    "longformer_seq_tensors_4096/train"),
    "cpu")

In [3]:
(token_ids, mention_ids, label_ids, attn_mask,
    global_attn_mask, doc_ids) = train_dataset.tensors

In [4]:
print(f"token_ids: dtype={token_ids.dtype}, shape={token_ids.shape}")
print(f"mention_ids: dtype={mention_ids.dtype}, shape={mention_ids.shape}")
print(f"label_ids: dtype={label_ids.dtype}, shape={label_ids.shape}")
print(f"attn_mask: dtype={attn_mask.dtype}, shape={attn_mask.shape}")
print(f"global_attn_mask: dtype={global_attn_mask.dtype}, "
        f"shape={global_attn_mask.shape}")
print(f"doc_ids: dtype={doc_ids.dtype}, shape={doc_ids.shape}")

token_ids: dtype=torch.int64, shape=torch.Size([34785, 4096])
mention_ids: dtype=torch.int32, shape=torch.Size([34785, 4096])
label_ids: dtype=torch.int64, shape=torch.Size([34785, 4096])
attn_mask: dtype=torch.float32, shape=torch.Size([34785, 4096])
global_attn_mask: dtype=torch.float32, shape=torch.Size([34785, 4096])
doc_ids: dtype=torch.int32, shape=torch.Size([34785])


In [5]:
n_mentions_per_label_sequence = []
for cluster_label_ids in label_ids[:100]:
    n_mentions_per_label_sequence.append((cluster_label_ids == 1).sum().item())
print("Number of mentions @ label sequence = "
        f"{collections.Counter(n_mentions_per_label_sequence)}")

Number of mentions @ label sequence = Counter({2: 54, 3: 27, 5: 7, 4: 4, 6: 2, 9: 2, 8: 1, 10: 1, 7: 1, 13: 1})


Check DataParallel

In [6]:
input_size = 5
output_size = 2

batch_size = 30
data_size = 100

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class RandomDataset(Dataset):

    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset(input_size, data_size),
                         batch_size=batch_size, shuffle=True)

class Model(nn.Module):
    # Our model

    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        output = self.fc(input)
        print("\tIn Model: input size", input.size(),
              "output size", output.size())

        return output

In [7]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
  model = nn.DataParallel(model)

model.to(device)

Let's use 2 GPUs!


DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [8]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
	In Model: input size torch.Size([15, 5]) output size torch.Size([15, 2])
Outside: input size torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
	In Model: input size torch.Size([5, 5]) output size torch.Size([5, 2])
Outside: input size torch.Size([10, 5]) output_size torch.Size([10, 2])
